In [1]:
import pandas as pd
import random
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np

## Load Tracking Results

In [2]:
df = pd.read_csv('tracker_results.csv')
df.head()

,frame_num,id,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax
0,0,-1,261.01556,813.442000,363.86530,1072.62330
1,0,-1,1614.05760,636.934940,1704.69370,837.91077
2,0,-1,883.42316,416.066000,966.37427,594.06540
3,0,-1,1456.64400,47.858955,1525.22530,168.81123
4,0,-1,1654.75800,133.837520,1710.17480,273.05527


Here we have all the keys pieces of information we need:
    * frame_num - Frame Number in the Video
    * id - Track ID to track a person across time
    * bbox_xmin, bbox_ymin, bbox_xmax, bbox_ymax - bounding box coordinates of the tracked box

### Run logic for a subset of frames

In [3]:
df= df[df.frame_num <=600]

## Create a dictionary to count total violations
track_list = df.id.unique()
track_violations = { i : [] for i in track_list }

## Measuring Distance between people

In [4]:
## Initialize two new violations
df['safe'] = 0
df['count_violations']=0

### Create Helper Functions

In [5]:
def distance_boxes(boxA, boxB):
    import math
    center_boxA = [(boxA[0] + boxA[2])/ 2.0, (boxA[1] + boxA[3])/2.0]
    center_boxB = [(boxB[0] + boxB[2])/ 2.0, (boxB[1] + boxB[3])/2.0]
    pixel_distance  = math.sqrt( ((center_boxA[0]-center_boxB[0])**2)+((center_boxA[1]-center_boxB[1])**2) )
    return pixel_distance

In [6]:
def get_bbox(frame_num, track):
    bbox_list = []
    data = df[(df.frame_num == frame_num) & (df.id == track)].reset_index()
    if data.shape[0] > 0:
        x1 = int(data.at[0, 'bbox_xmin'])
        bbox_list.append(x1)
        y1 = int(data.at[0, 'bbox_ymin'])
        bbox_list.append(y1)
        x2 = int(data.at[0, 'bbox_xmax'])
        bbox_list.append(x2)
        y2 = int(data.at[0, 'bbox_ymax'])
        bbox_list.append(y2)
        
    return bbox_list

### Main Function for measuring distance b/w players and highlighting unsafe behavior

In [7]:
def model_distancing(df,  proximity):

    print("Simulation Started")
    time1 = time.time()
    max_frame = df['frame_num'].max()
    for frame_num in range(3, max_frame):
        pairs_done = []
        if frame_num % 50 == 0:
            print("Completed: ", frame_num)
        ## Look at all tracks in this frame
        tracks_this_frame = list(df[df.frame_num==frame_num]['id'])
        ## Measure distance between this and every other track
        for track in tracks_this_frame:
            for other in tracks_this_frame:
                ## Don't match to yourself
                if track == other:
                    pass
                else:
                    this_pair = str(track) + "-" + str(other)
                    if this_pair not in pairs_done:
                        pixel_dist = distance_boxes(get_bbox(frame_num, track), get_bbox(frame_num, other))
                        ## Reverse pair
                        reverse_pair = str(other) + "-" + str(track)
                        ## Append both to pairs done to save time
                        pairs_done.append(this_pair)
                        pairs_done.append(reverse_pair)
                        if pixel_dist < proximity:
                            ## If Violation then both parties get unsafe behavior
                            index_tr = np.where((df.frame_num == frame_num) & (df.id == track))[0].tolist()
                            index_oth = np.where((df.frame_num == frame_num) & (df.id == other))[0].tolist()
                            df.set_value(index_tr[0], 'safe', 1)
                            df.set_value(index_oth[0], 'safe', 1)
                            
                            ## Increment count of violations
                            track_violations[track].append(other)
                            track_violations[other].append(track)
                            
            ## Append the total violations to data frame
            index_tr = np.where((df.frame_num == frame_num) & (df.id == track))[0].tolist()
            df.set_value(index_tr[0], 'count_violations', len(list(set(track_violations[track]))))
        
                       
    print("Simulation Complete")
    print("Time Taken: ", time.time() - time1)
    return df

In [8]:
proximity = 70

In [9]:
df = model_distancing(df, proximity)

Simulation Started


/home/priya/miniconda3/envs/tf13/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/priya/miniconda3/envs/tf13/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/priya/miniconda3/envs/tf13/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Completed:  50
Completed:  100
Completed:  150
Completed:  200
Completed:  250
Completed:  300
Completed:  350
Completed:  400
Completed:  450
Completed:  500
Completed:  550
Simulation Complete
Time Taken:  255.6870801448822


## Analyzing the results 

In [10]:
df.head()

,frame_num,id,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax,safe,count_violations
0,0,-1,261.01556,813.442000,363.86530,1072.62330,0,0
1,0,-1,1614.05760,636.934940,1704.69370,837.91077,0,0
2,0,-1,883.42316,416.066000,966.37427,594.06540,0,0
3,0,-1,1456.64400,47.858955,1525.22530,168.81123,0,0
4,0,-1,1654.75800,133.837520,1710.17480,273.05527,0,0


In [11]:
df['safe'].value_counts()

0    8599
1    1922
Name: safe, dtype: int64

In [12]:
df['count_violations'].value_counts()

0     4367
1     2344
2     1584
4      743
3      696
5      351
6      322
7       78
10      19
8       17
Name: count_violations, dtype: int64

## Visualize on a video

In [13]:
def get_safe(frame_num, track):
    data = df[(df.frame_num == frame_num) & (df.id == track)].reset_index()
    return int(data.at[0, 'safe']), int(data.at[0, 'count_violations'])

In [18]:
def create_visualization(video_path, df, proximity):
    vid_name = 'TownCenter_' + str(proximity)
    width = 1920
    height = 1080
    fps = 25
    file_name = os.path.join('videos_out', vid_name+"_dist"+".avi")
    vwriter = cv2.VideoWriter(file_name,
                          cv2.VideoWriter_fourcc(*'MJPG'),
                          fps, (width, height))
    
    ignore_tracks = [10, 14, 27, 20, 35, 42, 26, 16, 28]
    
    cap = cv2.VideoCapture(video_path)
    max_frame = df['frame_num'].max()
    for frame_num in range(3, max_frame):
        ret, image_np = cap.read()
#         image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
        tracks_this_frame = list(df[df.frame_num==frame_num]['id'])
        for track in tracks_this_frame:
            if track in ignore_tracks:
                pass
            else:
                coords = get_bbox(frame_num, track)
                safe_behavior, count_violations = get_safe(frame_num, track)
                ## Understand if this person is safe
                if safe_behavior == 0:
                    color = (0, 255, 0)
                else:
                    color = (0, 0, 255)
                    
                text = "Violation: " + str(count_violations)
                if count_violations == 0:
                    color_text = (0,0,0)
                else:
                    color_text = (0, 0, 255)
                cv2.rectangle(image_np, (coords[0], coords[1]) , (coords[2], coords[3]), color,2)
                cv2.putText(image_np, text, (coords[0]-8,coords[1]-8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text, 2, cv2.LINE_AA)
                ## Put track ID - optional
#                 cv2.putText(image_np, str(track), (coords[0]+5,coords[1]+5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA)

                
                if safe_behavior == 1:
                    cv2.putText(image_np, str("Too Close"), (coords[2]+ 5,coords[3]+ 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
       
        vwriter.write(image_np)
    vwriter.release()
    cap.release()
    print("Video Written to: ", file_name)

In [ ]:
create_visualization('videos/TownCentreXVID_small.mp4', df, proximity)